In [1]:
import numpy as np
import cv2
import os
from sklearn import svm

### Parameters

In [2]:
orange_training_dir = 'Training/Orange'
apple_training_dir = 'Training/Apple Red 1'
orange_testing_dir = 'Test/Orange'
apple_testing_dir = 'Test/Apple Red 1'

### Extract features from training data

In [3]:
XTrain = np.empty(3)
yTrain = int(np.empty(1))


for root, dirs, files in os.walk(orange_training_dir):
    dirs.sort()
    for filename in sorted(files):
        imagefilename = os.path.join(root, filename)
        image = cv2.imread(imagefilename)
        features = np.mean(np.mean(image, axis=0), axis=0)
        XTrain = np.vstack([XTrain, features])
        yTrain = np.hstack([yTrain, 0])
        
for root, dirs, files in os.walk(apple_training_dir):
    dirs.sort()
    for filename in sorted(files):
        imagefilename = os.path.join(root, filename)
        image = cv2.imread(imagefilename)
        features = np.mean(np.mean(image, axis=0), axis=0)
        XTrain = np.vstack([XTrain, features])
        yTrain = np.hstack([yTrain, 1])       

### Extract features from test data

In [4]:
XTest = np.empty(3)
yTest = int(np.empty(1))

for root, dirs, files in os.walk(orange_testing_dir):
    dirs.sort()
    for filename in sorted(files):
        imagefilename = os.path.join(root, filename)
        image = cv2.imread(imagefilename)
        features = np.mean(np.mean(image, axis=0), axis=0)        
        XTest = np.vstack([XTest, features])
        yTest = np.hstack([yTest, 0])
        
        
for root, dirs, files in os.walk(apple_testing_dir):
    dirs.sort()
    for filename in sorted(files):
        imagefilename = os.path.join(root, filename)
        image = cv2.imread(imagefilename)
        features = np.mean(np.mean(image, axis=0), axis=0)        
        XTest = np.vstack([XTest, features])
        yTest = np.hstack([yTest, 1])           

### Train and test model

In [5]:
# Train model
clf = svm.LinearSVC(C=1.)
clf.fit(XTrain/255, yTrain)

# Test model
clf.score(XTest/255, yTest)

0.9969230769230769

### Generate C code for prediction/inference

In [6]:
from sklearn_porter import Porter

porter = Porter(clf, language='c')
output = porter.export()

print(output)

#include <stdlib.h>
#include <stdio.h>
#include <math.h>

double coefficients[3] = {12.445481864888215, -5.805616230442312, -1.857350543112634};
double intercepts = -0.4491484726537361;

int predict (float features[]) {
    int i, l;
    double prob = 0.;
    for (i = 0, l = sizeof(coefficients) / sizeof (coefficients[0]); i < l; i++) {
        prob += coefficients[i] * features[i];
    }
    if (prob + intercepts > 0) {
        return 1;
    }
    return 0;
}

int main(int argc, const char * argv[]) {

    /* Features: */
    float features[argc-1];
    int i;
    for (i = 1; i < argc; i++) {
        features[i-1] = atof(argv[i]);
    }

    /* Prediction: */
    printf("%d", predict(features));
    return 0;

}
